In [1]:
import numpy as np
import wave
import os
import sys
import numpy as np
from os import listdir
from os.path import isfile, isdir, join
import wave
import struct
from tqdm import tqdm
import tensorflow as tf
import scipy.io as spio
import matplotlib.pyplot as plt
import json
# import soundfile as sf

In [2]:
max_int32_value = 2147483647
converted_training_mat_file = "../../NCTU/DNN_VAD/int_training_data/training_data_int32.mat"
originat_weight_mat = "../../Weights/w_original_128x32.mat"

In [3]:
def S(bits):
    return 2.0 ** (bits - 1)

In [4]:
def test_conversion():
    testary = np.array([[0.546846464, 0.96796461, 0.7496664],[0.48649646,0.32466841,0.5346434]])
    res_ary = performConvertion(testary, 1000000, 'uint32')
    print(res_ary)

In [5]:
# def load_weights(weightFile):
#     dict_data = spio.loadmat(weightFile)
#     w1 = dict_data["w1"]
#     w2 = dict_data["w2"]
#     w3 = dict_data["w3"]
#     b1 = dict_data["b1"]
#     b2 = dict_data["b2"]
#     b3 = dict_data["b3"]
#     return w1, w2, w3, b1, b2, b3

In [7]:
#for load training data
def load_training_data(fileName):
    ret_mat = spio.loadmat(fileName)["x_data"]
    return ret_mat

def performConvertion(src_array, scalar_multiplier, dtypeStr):
    #ret_clip = (audio_array[0:mat_len]*max_value).astype(dtype='double')
    processed_array = (src_array*scalar_multiplier).astype(dtypeStr)
    return processed_array

def performAdvnacedConvertion(src_array,bits,dtypeStr):
    scale = S(bits)
    w1,w2,w3,b1,b2,b3 = load_weights(src_array)
    quant_w1 = quant_weight(w1,8)
    print(quant_w1)
#     return processedWeight

def load_weights(weightFile):
    dict_data = spio.loadmat(weightFile)
    w1 = dict_data["w1"]
    w2 = dict_data["w2"]
    w3 = dict_data["w3"]
    b1 = dict_data["b1"]
    b2 = dict_data["b2"]
    b3 = dict_data["b3"]
    return w1, w2, w3, b1, b2, b3

In [20]:
def quant_weight(ori_weight, bits, typeStr):
    min_wt = np.amin(ori_weight)
    max_wt = np.amax(ori_weight)
    # calculat the bits necessary for representing the range
    repr_bits = int(np.ceil(np.log2(max(abs(min_wt),abs(max_wt)))))
    frac_bits = bits-repr_bits-1
    quant_weight = np.round(ori_weight*(2**frac_bits)).astype(typeStr)
    # we need to return the frac_bits for the reason that when we do the inference
    # we need to scale the quantized weights to their original range by
    # weight = quant_weight * 2^frac_bits
    return quant_weight, frac_bits

In [1]:
def performConversion(srcWeightFile, bits,typeStr,des_file):
    w1,w2,w3,b1,b2,b3 = load_weights(srcWeightFile)
    quant_w1, frac_bits1 = quant_weight(w1,bits,typeStr)
    quant_w2, frac_bits2 = quant_weight(w2,bits,typeStr)
    quant_w3, frac_bits3 = quant_weight(w3,bits,typeStr)
    
    quant_b1, frac_bits4 = quant_weight(b1,bits,typeStr)
    quant_b2, frac_bits5 = quant_weight(b2,bits,typeStr)
    quant_b3, frac_bits6 = quant_weight(b3,bits,typeStr)
    print("\nquant_w1 is\n{}".format(quant_w2))
    print("\nquant_scale for w1 is\n{}".format(frac_bits1))
    print("\nquant_scale for w2 is\n{}".format(frac_bits2))
    print("\nquant_scale for w3 is\n{}".format(frac_bits3))
    print("\nquant_scale for b1 is\n{}".format(frac_bits4))
    print("\nquant_scale for b2 is\n{}".format(frac_bits5))
    print("\nquant_scale for b3 is\n{}".format(frac_bits6))
    spio.savemat(des_file,{
                          'w1': quant_w1, 'w2': quant_w2, 'w3': quant_w3, 
                          'b1': quant_b1, 'b2': quant_b2, 'b3': quant_b3,
                          's1':frac_bits1, 's2':frac_bits2, 's3':frac_bits3,
                          's4':frac_bits4, 's5':frac_bits5, 's6':frac_bits6
                         })
    
    


In [24]:
save_file = "../../Weights/int8_weights_for_original_128x32_v3.mat"
performConversion(originat_weight_mat,8,"int8",save_file)


quant_w1 is
[[ 15   4  -7 ...  23  -3   2]
 [  8   3   4 ... -11 -13   5]
 [ -3   0  -6 ...  -5 -20  31]
 ...
 [ 11 -25   9 ...   8 -14  -4]
 [  0  -4   0 ...   0 -26 -18]
 [ -6   5   4 ...  10 -26  -8]]

quant_scale for w1 is
5

quant_scale for w2 is
6

quant_scale for w3 is
5

quant_scale for b1 is
6

quant_scale for b2 is
7

quant_scale for b3 is
8


In [18]:
def save_training_mat(srcArray):
    speech_mat_dict = {"x_data":srcArray}
    spio.savemat(converted_mat_file,speech_mat_dict,oned_as='column')
    
def save_weight_mat(srcWeightFile, bits,typeStr):
    weight1,weight2,weight3,bias1,bias2,bias3 = load_weights(srcWeightFile)
    weight1 = performConvertion(weight1,bits,typeStr)
    weight2 = performConvertion(weight2,bits,typeStr)
    weight3 = performConvertion(weight3,bits,typeStr)
    bias1 = performConvertion(bias1,bits,typeStr)
    bias2 = performConvertion(bias2,bits,typeStr)
    bias3 = performConvertion(bias3,bits,typeStr)
    print(weight2)
    spio.savemat("../../Weights/int{}_weights_for_original_128x32.mat".format(bits),
                     {'w1': weight1, 'w2': weight2, 'w3': weight3, 'b1': bias1, 'b2': bias2, 'b3': bias3})

In [19]:
save_weight_mat(converted_weight_mat)

[[  7   1  -3 ...  11  -1   1]
 [  4   1   2 ...  -5  -6   2]
 [ -1   0  -3 ...  -2 -10  15]
 ...
 [  5 -12   4 ...   4  -6  -2]
 [  0  -1   0 ...   0 -12  -9]
 [ -2   2   1 ...   4 -13  -3]]


In [40]:
nctu_training_mat = "../../speechData/nctu_origin_training_data/train_1106a_sharp_12.mat"
src_array = loaddata(nctu_training_mat)

In [41]:
print(src_array)

[[0.26684122 0.70283141 0.88426972 ... 0.71809787 0.16920831 0.16905722]
 [0.17060792 0.56428417 0.91814424 ... 0.75815144 0.05993171 0.0333237 ]
 [0.49442102 0.79770209 0.84977773 ... 0.84733115 0.0953902  0.        ]
 ...
 [0.7567909  0.99432799 0.70640176 ... 0.22294714 0.09880513 0.07683635]
 [0.76510084 0.99751292 0.69988347 ... 0.11805857 0.04398413 0.03533766]
 [0.75494277 0.99998792 0.6995279  ... 0.09852559 0.         0.01161164]]


In [42]:
converted_ary = performConvertion(src_array,max_int32_value,"int32")

In [43]:
print(converted_ary)

[[ 573037146 1509318960 1898954757 ... 1542103438  363372077  363047614]
 [ 366377713 1211791028 1971699742 ... 1628117816  128702374   71562108]
 [1061761056 1713052187 1824883783 ... 1819629791  204848902          0]
 ...
 [1625196090 2135303087 1516986224 ...  478775340  212182402  165004796]
 [1643041537 2142142687 1502988301 ...  253528839   94455201   75887043]
 [1621227245 2147457716 1502224725 ...  211582086          0   24935799]]


In [44]:
save_to_mat(converted_ary)